# Feature Selection

In the last few lectures, we learned how to use hold-out "test" sets and cross-validation to gain appropriate estimates of a model's performance on unseen data. There, the focus was on choosing a good "complexity" parameter, such as the depth of a decision tree. In this lecture, we'll instead show how to use cross-validation to get an estimate of which columns in the data should or should not be included in a model. It's very common in practice that not all columns will be used in the best model, and many, many machine learning reseachers devote their careers to studying the problem of how to intelligently and automatically choose only the most relevant columns for models. In the literature, this problem is usually called *feature selection*. In this lecture, we'll take a quick look at how feature selection can improve model performance. 

For this demonstration, we'll switch from decision trees to logistic regression. Logistic regression is a form of regression modeling well-suited for predicting probabilities and class labels. 

Let's begin by running some familiar blocks of code, in which we load our core libraries, read in the data, split the data, and clean the data. 

In [1]:
#standard imports
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

#read in data
titanic=pd.read_csv("titanic.csv")


We are going to divide the data into a training and testing sets and prepocess it by changing male/female to 1/0 and dropping the name column. This code is nearly exactly the same as earlier videos.

In [2]:
from sklearn.model_selection import train_test_split

np.random.seed(1111)
train, test = train_test_split(titanic, test_size = 0.2)

In [3]:
from sklearn import preprocessing
def prep_titanic_data(data_df):
    df = data_df.copy()
    le = preprocessing.LabelEncoder()
    df['Sex'] = le.fit_transform(df['Sex'])
    df = df.drop(['Name'], axis = 1)
    
    X = df.drop(['Survived'], axis = 1)
    y = df['Survived']
        
    return(X, y)

X_train, y_train = prep_titanic_data(train)
X_test,  y_test  = prep_titanic_data(test)

Using logistic regression is almost exactly the same as using the decision tree classifier. Let's go ahead and use cross-validation to estimate the predictive performance of the model. 

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

LR=LogisticRegression()
cross_val_score(LR,X_train,y_train,cv=5).mean()

0.7940365597842375

Is this the best we can do? If you've studied logistic regression before, you may know that using lots of columns doesn't always help -- due to *multicollinearity*, the model's predictive performance can actually suffer. This is actually another aspect of *overfitting*. Adding more columns makes the model more flexible, and we've seen that that is not always beneficial. So, a natural question is whether we can achieve the same (or better?) model performance by using only a subset of the columns. 


It's easy to train a model on a subset of the data. For example: 

In [5]:
#all the columns except fare
cols = ['Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard']
print("training with columns "+str(cols))

LR=LogisticRegression()
cross_val_score(LR,X_train[cols],y_train,cv=5).mean()

training with columns ['Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard']


0.8025072420337629

Interesting! Excluding the last column (Fare) actually improved our CV score slightly. 

## Systematic Feature Selection

Now, let's write a function that will let us do this systematically. Our function will use cross-validation to avoid "peeking" at the test set. 

In [8]:
def check_column_score(cols):
    """
    Trains and evaluates the model via crossvalidation on the columns
    of the dataset with select indeces
    """
    print("training with columns "+str(cols))

    LR=LogisticRegression()
    return cross_val_score(LR,X_train[cols],y_train,cv=5).mean()

In [9]:
combos = [['Sex', 'Age', 'Fare'],
          ['Pclass', 'Sex', 'Age'],
          ['Pclass', 'Parents/Children Aboard'],
          ['Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard'],
          ['Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']]

for combo in combos:
    x=check_column_score(combo)
    print("CV score is "+str(np.round(x,3)))


training with columns ['Sex', 'Age', 'Fare']
CV score is 0.773
training with columns ['Pclass', 'Sex', 'Age']
CV score is 0.795
training with columns ['Pclass', 'Parents/Children Aboard']
CV score is 0.671
training with columns ['Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard']
CV score is 0.803
training with columns ['Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']
CV score is 0.794


-  Which combo scored the best?
-  Which combo scored the worst?

Now, lets check on the test set

In [10]:
def test_column_score(cols):
    """
    Test model performance on the data with selected columns
    """
    
    print("training with columns "+ str(cols))
    LR=LogisticRegression()
    LR.fit(X_train[cols],y_train)
    
    return LR.score(X_test[cols],y_test)

In [12]:
for combo in combos:
    x=test_column_score(combo)
    print("The test score is "+ str(np.round(x,3)))

training with columns ['Sex', 'Age', 'Fare']
The test score is 0.82
training with columns ['Pclass', 'Sex', 'Age']
The test score is 0.803
training with columns ['Pclass', 'Parents/Children Aboard']
The test score is 0.742
training with columns ['Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard']
The test score is 0.831
training with columns ['Pclass', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']
The test score is 0.82


Indeed, we achieved a higher prediction score on the test set by ignoring the "Fare" column completely. 

There are a number of sophisticated algorithms for automated feature selection, but we won't go further into this topic in this course. 